In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')

In [2]:

from keras.layers import Input, Lambda, Dense, Flatten
from keras.models import Model
#from keras.applications.resnet50 import ResNet50
from keras.applications.vgg16 import VGG16
from keras.applications.vgg16 import preprocess_input
from keras.preprocessing import image
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
import numpy as np
from glob import glob
import matplotlib.pyplot as plt

In [3]:
# re-size all the images to this
IMAGE_SIZE = [224, 224]

In [4]:
train_path= r"C:\Users\user\Desktop\Data sets\dog_dataset\training_set\training_set"
test_path=r"C:\Users\user\Desktop\Data sets\dog_dataset\test_set\test_set"

In [5]:
# Import the Vgg 16 library as shown below and add preprocessing layer to the front of VGG
# Here we will be using imagenet weights

vgg = VGG16(input_shape=IMAGE_SIZE + [3], weights='imagenet', include_top=False)

In [6]:
#don't train existing weights
for layer in vgg.layers:
    layer.trainable=False


In [7]:
# useful for getting number of output classes
folders = glob(r'C:\Users\user\Desktop\Data sets\dog_dataset\training_set\training_set\*')

In [8]:
folders

['C:\\Users\\user\\Desktop\\Data sets\\dog_dataset\\training_set\\training_set\\cats',
 'C:\\Users\\user\\Desktop\\Data sets\\dog_dataset\\training_set\\training_set\\dogs']

In [9]:

# our layers - you can add more if you want
x = Flatten()(vgg.output)

In [10]:
prediction = Dense(len(folders), activation='softmax')(x)

# create a model object
model = Model(inputs=vgg.input, outputs=prediction)

In [11]:
prediction

<KerasTensor: shape=(None, 2) dtype=float32 (created by layer 'dense')>

In [12]:
model

In [13]:
len(folders)

2

In [14]:
# view the structure of the model
model.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 224, 224, 3)]     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0     

In [15]:
# tell the model what cost and optimization method to use
model.compile(
  loss='categorical_crossentropy',
  optimizer='adam',
  metrics=['accuracy']
)

In [16]:
# Use the Image Data Generator to import the images from the dataset
from keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(rescale = 1./255,
                                   shear_range = 0.2,
                                   zoom_range = 0.2,
                                   horizontal_flip = True)

test_datagen = ImageDataGenerator(rescale = 1./255)

In [17]:
# Make sure you provide the same target size as initialied for the image size
training_set = train_datagen.flow_from_directory(r'C:\Users\user\Desktop\Data sets\dog_dataset\training_set\training_set',
                                                 target_size = (224, 224),
                                                 batch_size = 32,
                                                 class_mode = 'categorical')

Found 8005 images belonging to 2 classes.


In [18]:
testing_set=test_datagen.flow_from_directory(r"C:\Users\user\Desktop\Data sets\dog_dataset\test_set\test_set",
                                            target_size=(224,224),
                                            batch_size=32,
                                            class_mode='categorical')

Found 2023 images belonging to 2 classes.


In [19]:
# Run the cell. It will take some time to execute
r = model.fit_generator(training_set,
  validation_data=testing_set,
  epochs=1)

  2/251 [..............................] - ETA: 1:00:18 - loss: 1.8162 - accuracy: 0.3906

KeyboardInterrupt: 

# I'm intrupting my keyboard because it takes more time to execute 